In [1]:
!mkdir server
%cd server
!npm init -y
!npm install express mongoose body-parser

c:\project_YJ\dev\projects\front\temperary_project\server


In [3]:
%%writefile "tsconfig.json"
{
  "compilerOptions": {
    "target": "ES6",
    "module": "commonjs",
    "outDir": "./dist",
    "rootDir": "./src",
    "strict": true,
    "esModuleInterop": true,
    "skipLibCheck": true,
    "forceConsistentCasingInFileNames": true
  }
}


In [ ]:
!npm install --save-dev typescript @types/node @types/express @types/body-parser ts-node
!mkdir src/config
!mkdir src/models
!mkdir src/controllers
!mkdir src/routes

In [4]:
%%writefile "src/config/db.ts"
import mongoose from "mongoose";

const connectDB = async () => {
  try {
    await mongoose.connect("mongodb://localhost:27017/mydatabase");
    console.log("MongoDB connected");
  } catch (error) {
    console.error("MongoDB connection error:", error);
    process.exit(1);
  }
};

export default connectDB;


In [5]:
%%writefile "src/models/User.ts"
import mongoose, { Schema, Document } from "mongoose";

export interface IUser extends Document {
  name: string;
  email: string;
}

const UserSchema: Schema = new Schema({
  name: { type: String, required: true },
  email: { type: String, required: true },
});

export default mongoose.model<IUser>("User", UserSchema);


In [8]:
%%writefile "src/routes/userRoutes.ts"
import { Router } from "express";
import { createUser, getUsers } from "../controllers/userController";

const router = Router();

router.post("/users", createUser);
router.get("/users", getUsers);

export default router;


In [7]:
%%writefile "src/controllers/userController.ts"
import { Request, Response } from "express";
import User from "../models/User";

export const createUser = async (req: Request, res: Response) => {
  const { name, email } = req.body;
  const newUser = new User({ name, email });

  try {
    const savedUser = await newUser.save();
    res.json(savedUser);
  } catch (err) {
    res.status(500).send(err);
  }
};

export const getUsers = async (req: Request, res: Response) => {
  try {
    const users = await User.find();
    res.json(users);
  } catch (err) {
    res.status(500).send(err);
  }
};


In [9]:
%%writefile "src/index.ts"
import express from "express";
import bodyParser from "body-parser";
import path from "path";
import connectDB from "./config/db";
import userRoutes from "./routes/userRoutes";

const app = express();
const port = 3000;

// MongoDB 연결 설정
connectDB();

// 미들웨어 설정
app.use(bodyParser.json());
app.use(express.static(path.join(__dirname, "../../client/build")));

// API 라우트 설정
app.use("/api", userRoutes);

// 모든 기타 경로에 대해 React 애플리케이션의 index.html 반환
app.get("*", (req, res) => {
  res.sendFile(path.join(__dirname, "../../client/build", "index.html"));
});

app.listen(port, () => {
  console.log(`Server is running at http://localhost:${port}`);
});


In [10]:
%%writefile "package.json"
{
  "name": "server",
  "version": "1.0.0",
  "main": "index.js",
  "scripts": {
    "start": "ts-node src/index.ts"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "description": "",
  "dependencies": {
    "body-parser": "^1.20.2",
    "express": "^4.19.2",
    "mongoose": "^8.5.2"
  },
  "devDependencies": {
    "@types/body-parser": "^1.19.5",
    "@types/express": "^4.17.21",
    "@types/node": "^22.0.0",
    "ts-node": "^10.9.2",
    "typescript": "^5.5.4"
  }
}


In [12]:
%cd ..
!npx create-react-app client --template typescript
%cd client/

c:\project_YJ\dev\projects\front\temperary_project\client


In [13]:
%%writefile "src/App.tsx"
import React, { useState, useEffect } from 'react';

interface IUser {
  _id: string;
  name: string;
  email: string;
}

const App: React.FC = () => {
  const [name, setName] = useState<string>('');
  const [email, setEmail] = useState<string>('');
  const [users, setUsers] = useState<IUser[]>([]);

  const fetchUsers = async () => {
    try {
      const response = await fetch('/api/users');
      const data = await response.json();
      setUsers(data);
    } catch (error) {
      console.error('Error fetching users:', error);
    }
  };

  useEffect(() => {
    fetchUsers();
  }, []);

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();
    const newUser = { name, email };

    try {
      const response = await fetch('/api/users', {
        method: 'POST',
        headers: {
          'Content-Type': 'application/json',
        },
        body: JSON.stringify(newUser),
      });
      const data = await response.json();
      setUsers([...users, data]);
      setName('');
      setEmail('');
    } catch (error) {
      console.error('Error adding user:', error);
    }
  };

  return (
    <div>
      <h1>User Form</h1>
      <form onSubmit={handleSubmit}>
        <input
          type="text"
          placeholder="Name"
          value={name}
          onChange={(e) => setName(e.target.value)}
          required
        />
        <input
          type="email"
          placeholder="Email"
          value={email}
          onChange={(e) => setEmail(e.target.value)}
          required
        />
        <button type="submit">Submit</button>
      </form>

      <h2>Users</h2>
      <ul>
        {users.map((user) => (
          <li key={user._id}>{user.name} ({user.email})</li>
        ))}
      </ul>
    </div>
  );
};

export default App;


In [ ]:
!npm run build
%cd ..